In [33]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import face_recognition_models
import dlib
import os
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates

In [34]:
face_recognition_model = face_recognition_models.face_recognition_model_location()
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)

In [36]:
mp_drawing = mp.solutions.drawing_utils
mp_face_detect = mp.solutions.face_detection
face_detect = mp_face_detect.FaceDetection(min_detection_confidence=0.7)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1,min_detection_confidence=0.5)

In [37]:
def raw_landmark(filepath,file):
    face_image = cv2.imread(filepath)
    #print(np.max(face_image))
    width,height=face_image.shape[:-1]
    image_input = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
    results = face_mesh_images.process(image_input)
    if results.multi_face_landmarks:
        for i,face_landmarks in enumerate(results.multi_face_landmarks): 
            if i==0:
                raw_landmark_set = []
                for index in landmark_points_5:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    raw_landmark_set.append((x, y))
                #encodings=face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters=1)
                raw_landmark_set=np.array(raw_landmark_set)
                row =pd.DataFrame({"filename":file,"landmarks": [raw_landmark_set]})
        return row

In [38]:
#p="E:\\Pix\\Munmun_wedding"
#p="C:\\Users\\jhash\\OneDrive - City University of New York\\GenArt\\ML\\Mandiberg\\images forward"
p="/Users/michaelmandiberg/Dropbox/facemap_dropbox/images forward"

In [39]:
curr=0
image_data=pd.DataFrame()
for subdir, dirs, files in os.walk(p):
    total = len(files)
    for file in files:
        curr+=1
        filepath = os.path.join(subdir, file)
        #print("File: %s" % filepath)
        print(curr,"/",total)
        filepath=filepath.replace('\\' , '/')
        data=raw_landmark(filepath,file)
        image_data = pd.concat([image_data,data],ignore_index=True)
print("Done")

1 / 258


NameError: name 'face_mesh_images' is not defined

In [25]:
print(image_data.tail())

Empty DataFrame
Columns: []
Index: []


In [28]:
filepath="/Users/michaelmandiberg/Dropbox/facemap_dropbox/images forward/faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct261_1.jpg"
filepath=filepath.replace('\\' , '/')
face_image = cv2.imread(filepath)
width,height=face_image.shape[:-1]
image_input = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
results = face_mesh_images.process(image_input)
if results.multi_face_landmarks:
    for i,face_landmarks in enumerate(results.multi_face_landmarks): 
        if i==0:
            raw_landmark_set = []
            for index in landmark_points_5:
                x = int(face_landmarks.landmark[index].x * width)
                y = int(face_landmarks.landmark[index].y * height)
                landmark_point=dlib.point([x,y])
                raw_landmark_set.append(landmark_point)
                #display(landmark_point)
            all_points=dlib.points(raw_landmark_set)
            display(all_points)
            #landmark_point=dlib.point(raw_landmark_set)
            #encodings=face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters=1)


NameError: name 'face_mesh_images' is not defined

In [29]:
def landmark_pt_list(mesh_results,width,height):
    landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
    landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
    landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]

    if mesh_results.multi_face_landmarks:
        for i,face_landmarks in enumerate(mesh_results.multi_face_landmarks): 
            if i==0:
                raw_landmark_set = []
                for index in landmark_points_5_1:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    landmark_point=dlib.point([x,y])
                    raw_landmark_set.append(landmark_point)
                    #display(landmark_point)
                all_points=dlib.points(raw_landmark_set)
#         return dlib.points([{
#             "nose_tip": [raw_landmark_set[0]],
#             "left_eye": raw_landmark_set[1:3],
#             "right_eye": raw_landmark_set[3:],
#         }])
        return all_points

def bounding_rect(detection_results,width,height):
    if detection_results.detections:
        for i,detection in enumerate(detection_results.detections):
            if i==0:
                # bbox data
                bbox = detection.location_data.relative_bounding_box
                xy_min = _normalized_to_pixel_coordinates(bbox.xmin, bbox.ymin, height,width)
                xy_max = _normalized_to_pixel_coordinates(bbox.xmin + bbox.width, bbox.ymin + bbox.height,height,width)
                if xy_min is None or xy_max is None:
                    #print("face out of frame")
                    return
                else:
                    xmin,ymin =xy_min
                    xmax,ymax = xy_max
                    #bbox_points = { "xmin" : xmin,"ymin" : ymin,"xmax" : xmax,"ymax" : ymax}
                    rectangle= dlib.rectangle(left=xmin, top=ymax, right=xmax, bottom=ymin)
                    return rectangle

In [30]:
# filepath="C:\\Users\\jhash\\Pictures\\images forward\\faceimg_crop1_X-20toX1_Y-4toY4_Z-3toZ3_maxResize0.5_ct261_1.jpg"
filepath="/Users/michaelmandiberg/Documents/GitHub/facemap/DSC_0492.jpg"
filepath=filepath.replace('\\' , '/')
face_image = cv2.imread(filepath)
width,height=face_image.shape[:-1]
image_input = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
detection_results = face_detect.process(image_input)
mesh_results = face_mesh.process(image_input)

all_points=  landmark_pt_list(mesh_results)  
b_box=bounding_rect(detection_results)
raw_landmark_set=dlib.full_object_detection(b_box,all_points)
display(all_points)
display(b_box)
encodings=face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters=1)


TypeError: landmark_pt_list() missing 2 required positional arguments: 'width' and 'height'

In [31]:
def ret_encoding(filepath,num_jitters=1):
    face_image = cv2.imread(filepath)
    width,height=face_image.shape[:-1]
    image_input = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
    detection_results = face_detect.process(image_input)
    mesh_results = face_mesh.process(image_input)
    
    all_points=  landmark_pt_list(mesh_results,width,height)  
    b_box=bounding_rect(detection_results,width,height)
    if (all_points is None) or (b_box is None):
        return 
    raw_landmark_set=dlib.full_object_detection(b_box,all_points)
    #display(all_points)
    #display(b_box)
    encodings=face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters)

    return encodings

In [32]:
file1="Test\\test11.jpg".replace('\\' , '/')
#file1="E:\\Pix\\Munmun_wedding\\DSC_0524.jpg".replace('\\' , '/')

enc1=(ret_encoding(file1))
dist=[]
for i in range(6,14):
    #file2="E:\\Pix\\Munmun_wedding\\DSC_05"+str(i)+".jpg".replace('\\' , '/')
    file2="Test\\test"+str(i)+".jpg".replace('\\' , '/')
    enc2=(ret_encoding(file2))
    if (enc1 is not None) and (enc2 is not None):
        print(file2)
        enc1=np.array(enc1)
        enc2=np.array(enc2)
        d=np.linalg.norm(enc1 - enc2, axis=0)
        print(d)
        dist.append(d)
        

[ WARN:0@443.504] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('Test/test11.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [21]:
file1="E:\\Pix\\Munmun_wedding\\DSC_0524.jpg".replace('\\' , '/')
enc1=(ret_encoding(file1))
dist=[]
for i in range(22,54):
    file2="E:\\Pix\\Munmun_wedding\\DSC_05"+str(i)+".jpg".replace('\\' , '/')
    enc2=(ret_encoding(file2))
    if (enc1 is not None) and (enc2 is not None):
        print(file2)
        enc1=np.array(enc1)
        enc2=np.array(enc2)
        d=np.linalg.norm(enc1 - enc2, axis=0)
        print(d)
        dist.append(d)
        

[ WARN:0@308.211] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('E:/Pix/Munmun_wedding/DSC_0524.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [131]:
dist

[0.26394233980185616,
 0.33006268519974874,
 0.0,
 0.4103105560215812,
 0.41458072989933475,
 0.4652770685378682,
 0.4519844876793332,
 0.30556772392866643,
 0.44270787650444055,
 0.3277119208900013,
 0.49255133781418725,
 0.3257423417556279,
 0.2902058595612748,
 0.5471849159296998,
 0.3642091641638166,
 0.5382755533872039,
 0.39189821819581516,
 0.3484603675189382,
 0.39012534906473467]

In [152]:
#file1="E:\\Pix\\Munmun_wedding\\DSC_0492.jpg".replace('\\' , '/')
file1="E:\\Pix\\Munmun_wedding\\DSC_0275.jpg".replace('\\' , '/')

enc1=(ret_encoding(file1))
dist=[]
#file2="E:\\Pix\\Munmun_wedding\\DSC_0446.jpg".replace('\\' , '/')
file2="E:\\Pix\\Munmun_wedding\\DSC_0525.jpg".replace('\\' , '/')

print(file2)
enc2=(ret_encoding(file2))
#display(enc2)
if (enc1 is not None) and (enc2 is not None):
    enc1=np.array(enc1)
    enc2=np.array(enc2)
    dist.append(np.linalg.norm(enc1 - enc2, axis=0))
dist

E:/Pix/Munmun_wedding/DSC_0525.jpg


[0.4771382735255576]

In [125]:
dist

[0.5130601053858391]

In [112]:
c=b
if not c :
    print("full")
else:
    print("empty")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [117]:
enc1

dlib.vector([-0.0804747, 0.0910025, 0.0769911, -0.0480331, -0.0944251, -0.0489447, -0.10361, -0.00703891, 0.132111, -0.0513232, 0.235515, 0.0362993, -0.220011, -0.039243, -0.041553, 0.101616, -0.126476, -0.0738419, -0.113043, -0.119332, 0.0343449, -0.0186584, 0.00066364, -0.00170676, -0.131435, -0.259795, -0.0797831, -0.106187, 0.0900066, -0.148663, 0.0150025, 0.0363624, -0.12216, -0.0129467, -0.027539, 0.0263516, -0.000913745, -0.0790531, 0.158763, -0.0156388, -0.133456, -0.0324634, -0.0366673, 0.277406, 0.194996, 0.0377514, 0.0654894, -0.0647332, 0.0343912, -0.270923, 0.0368137, 0.153475, 0.020433, 0.0679717, 0.0878614, -0.178231, 0.0258942, 0.0877184, -0.129055, 0.0236241, 0.00916781, -0.0573456, -0.0709152, -0.133017, 0.253069, 0.128938, -0.100669, -0.12534, 0.194731, -0.132965, -0.0291884, 0.073089, -0.121139, -0.135718, -0.217702, 0.0596202, 0.375289, 0.1815, -0.16883, 0.049299, -0.121696, -0.0388345, 0.0173798, 0.0133728, -0.0983486, -0.00393279, -0.0624877, 0.0656582, 0.193343,